In [1]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

import tqdm

import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
import seaborn as sns

from transformers import Trainer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments


/opt/anaconda3/envs/RoBERTa2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())

True
True


In [3]:
import torch
import pandas as pd
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    AutoModelForSequenceClassification, 
    TrainingArguments, 
    Trainer,
    EarlyStoppingCallback,
    DataCollatorWithPadding)

#import bitsandbytes as bnb

#import evaluate
import numpy as np

import random

In [4]:
access_token = 'ß'
mps_device = torch.device("mps")

In [5]:
from huggingface_hub import login
login(access_token, add_to_git_credential=True)

Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/jeahyukjeong/.cache/huggingface/token
Login successful


In [6]:
# 데이터 불러오기

data_total = pd.read_csv('./output/QCHAT_T_ASD_total_First.csv')
data_total=data_total[['Class/ASD Traits','text']] # 데이터셋에서 Class/ASD Traits 와 text column만 사용

# target label encoding
Q_ASD_mapping = {
    'No': 0, # TD
    'Yes': 1, # ASD
}

num_mapping = {
    0: 'No', # TD
    1: 'Yes', # ASD

}

data_total['encoded_label'] = data_total['Class/ASD Traits'].replace(Q_ASD_mapping)

# 결과
print("QCHAT label_encoded counts: ", data_total.encoded_label.value_counts())
data_total.head()

QCHAT label_encoded counts:  encoded_label
1    110
0     44
Name: count, dtype: int64


,Class/ASD Traits,text,encoded_label
0,0,", , , , , , , , ,",0
1,0,", , , , , , , , ,",0
2,0,", , , , , , , , ,",0
3,0,", , , , , , , , ,",0
4,0,", , , , , , , , ,",0


In [7]:
# 시드값 설정

if torch.cuda.is_available(): 
    print("GPU is available")

#set random seed 
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    # torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)
    
random_seed = 42
set_seed(random_seed)
SEED = 42

In [8]:
# X: input variables, y: target variable
X = data_total.drop('encoded_label', axis=1)
y = data_total['encoded_label']

# train-test split(7:3)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

train_and_validation_df = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
test_df = pd.concat([X_test, y_test], axis=1).reset_index(drop=True) # test dataset

# train-validation split(8:2)

# X: input variables, y: target variable
X = train_and_validation_df.drop('encoded_label', axis=1)
y = train_and_validation_df['encoded_label']

# Split the training data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

train_df = pd.concat([X_train, y_train], axis=1).reset_index(drop=True) # training dataset
valid_df = pd.concat([X_valid, y_valid], axis=1).reset_index(drop=True) # validation dataset

In [9]:
print("size of train dataset: ", len(train_df))
print("size of validation dataset: ", len(valid_df))
print("size of test dataset: ", len(test_df))

size of train dataset:  85
size of validation dataset:  22
size of test dataset:  47


In [10]:
# Tokenization에 사용될 모델 
tokenizer = AutoTokenizer.from_pretrained('google/gemma-2-2b')

# train set tokenization
encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True, max_length=32)
train_labels = train_df['encoded_label'].tolist()
train_input_ids = encodings['input_ids']
train_attention_masks = encodings['attention_mask']

# validation set tokenization
encodings = tokenizer(valid_df['text'].tolist(), truncation=True, padding=True, max_length=32)
valid_labels = valid_df['encoded_label'].tolist()
valid_input_ids = encodings['input_ids']
valid_attention_masks = encodings['attention_mask']

# test set tokenization
encodings = tokenizer(test_df['text'].tolist(), padding=True, truncation=True, return_tensors="pt")
test_input_ids = encodings['input_ids']
test_attention_masks = encodings['attention_mask']

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [11]:
# Before
print("text example from train BEFORE tokenization: ")
print(train_df['text'][0])

text example from train BEFORE tokenization: 
 , , , , , , , , ,play impairment stares blankly at play objects rather than manipulating them


In [12]:
# After
print("text example from train AFTER tokenization: ")
print("Input Ids: ", train_input_ids[0])
print("attention mask: ", train_attention_masks[0])

text example from train AFTER tokenization: 
Input Ids:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1688, 1688, 1688, 1688, 1688, 1688, 1688, 1688, 1688, 1752, 61551, 124686, 15601, 605, 696, 1554, 9113, 4644, 1178, 111076, 1174]
attention mask:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [13]:
# training에 사용하기 위해 데이터 타입 전환

# Training에 사용될 데이터 class
class QCHATDataset(torch.utils.data.Dataset):

    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __getitem__(self, idx):
        
        # training 과정에서 데이터가 사용될 때 tensor 타입으로 사용되도록 합니다. 
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.int),
            'attention_mask': torch.tensor(self.attention_masks[idx], dtype=torch.int),
            'labels': torch.tensor(self.labels[idx], dtype=torch.int)
        }

    def __len__(self):
        return len(self.labels)


# train set
train_dataset = QCHATDataset(train_input_ids, train_attention_masks, train_labels)
# Validation set
validation_dataset = QCHATDataset(valid_input_ids, valid_attention_masks, valid_labels)
# test dataset
test_dataset = QCHATDataset(test_input_ids, test_attention_masks, test_df['encoded_label'].tolist())

In [14]:
torch.int

torch.int32

In [15]:
#https://github.com/huggingface/transformers/issues/17971
#os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
"""
import torch
from transformers import TrainingArguments


class TrainingArgumentsWithMPSSupport(TrainingArguments):

    @property
    def device(self) -> torch.device:
        if torch.cuda.is_available():
            return torch.device("cuda")
        elif torch.backends.mps.is_available():
            return torch.device("mps")
        else:
            return torch.device("cpu")

"""

'\nimport torch\nfrom transformers import TrainingArguments\n\n\nclass TrainingArgumentsWithMPSSupport(TrainingArguments):\n\n    @property\n    def device(self) -> torch.device:\n        if torch.cuda.is_available():\n            return torch.device("cuda")\n        elif torch.backends.mps.is_available():\n            return torch.device("mps")\n        else:\n            return torch.device("cpu")\n\n'

In [16]:
import deepspeed

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ['TORCH_USE_CUDA_DSA']='1'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# RoBERTa Classification 기본 모델을 불러옵니다.
model = AutoModelForSequenceClassification.from_pretrained(
    "google/gemma-2-2b", 
    num_labels=2,
    #device_map = 'auto',
    #id2label = Q_ASD_mapping,
    #label2id = num_mapping
    )


# Training에 사용될 hyperparameter을 설정해줍니다.
training_args = TrainingArguments(
    output_dir='./output', #         # output directory for model checkpoints
    num_train_epochs=5,              # number of training epochs
    per_device_train_batch_size = 1,  # batch size for training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=0,                # number of warmup steps
    weight_decay=1e-8,               # strength of weight decay
    logging_dir='./Gemma/log',            # directory for storing logs
    logging_steps=10,              # log saving step. 100
    evaluation_strategy="steps",     # evaluate each `logging_steps`,
    learning_rate=2e-5,              # learning rate
    #gradient_accumulation_steps=8,
    #deepspeed="./deepspeed.json",
    #fp16=False,
    #save_steps=100,
)

[2024-09-09 13:11:09,980] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to mps (auto detect)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
W0909 13:11:10.448404 8332671680 torch/distributed/elastic/multiprocessing/redirects.py:27] NOTE: Redirects are currently not supported in Windows or MacOs.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  8.86it/s]
Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at google/gemma-2-2b and ar

In [17]:
# Training에 사용할 evaluation metrics를 설정해줍니다.

def compute_metrics_multiclass(p, num_classes=2):  # num_classes는 예시 값입니다. 실제 클래스 수로 변경하세요.
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    # Multiclass
    f1 = f1_score(labels, predictions, average='macro')
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)

    # Treat each class as a binary classification
    labels_binarized = label_binarize(labels, classes=range(num_classes))
    predictions_binarized = label_binarize(predictions, classes=range(num_classes))

    # Multiclass AUROC , Average Precision
    auroc = roc_auc_score(labels_binarized, predictions_binarized, multi_class='ovr')
    avgprc = average_precision_score(labels_binarized, predictions_binarized)

    return {
        "acc": acc, 
        "f1": f1, 
        "precision": precision, 
        "recall": recall, 
        "auroc": auroc, 
        "avgprc": avgprc
    }

def compute_metrics_wrapper(p):
    return compute_metrics_multiclass(p, num_classes=2) #0,1


In [18]:
# Training에 필요한 데이터셋, model, evaluation_metrics, hyperparameter 등을 주어줍니다.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics_wrapper  # We will define compute_metrics function later
)

# Training을 시작합니다.
trainer.train()


  0%|          | 0/425 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Evaluation 
trainer.evaluate()